### Install the libraries

In [ ]:
! pip install llama_index


In [ ]:
! pip install llama-index-llms-huggingface


In [ ]:
! pip install llama-index-llms-huggingface-api


In [ ]:
! pip install llama-index-embeddings-huggingface


In [ ]:
! pip install vllm

In [ ]:
!pip install -U llama-index llama-index-llms-groq groq

In [ ]:
!pip uninstall -y pinecone-client

!pip install pinecone

In [ ]:
#!pip install llama-index-core llama-index-vector-stores-pinecone pinecone-client
!pip install llama-index-core llama-index-vector-stores-pinecone


In [ ]:
!pip install --no-cache-dir pinecone

### Import the libraries

In [ ]:
from llama_index.core import SimpleDirectoryReader, Settings, StorageContext, VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
import pinecone
from pinecone import ServerlessSpec

### Load the documents

In [ ]:
docs = SimpleDirectoryReader("./papers").load_data()

In [ ]:
print (len(docs))

# 129 => SimpleDirectoryReader splits the documents into chunks

179


### Setting up the LLM => gives context

In [ ]:
groq_api_key = ''

llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    temperature=0
)

# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     context_window=2048,       # how much text the model can process at once
#     max_new_tokens=256,        # max number of tokens to generate in a response
#     device_map="auto"          # automatically use GPU if available
# )

Settings.llm=llm

### Setting up the embedder model => convert text to numerical representation

In [ ]:

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)
Settings.embed_model = embed_model

### Initialise Pinecone

In [ ]:
pc = pinecone.Pinecone(api_key="")

### Indexing

In [ ]:
index_name = "llamaindex-demo"

# Check if the index already exists, otherwise create one
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to the index just created
pinecone_index = pc.Index(index_name)

### Create the Pinecone vector store for LlamaIndex

In [ ]:
# Persist the index to a storage folder
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)


### Create a storage context

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

### Build the index

In [ ]:
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    show_progress=True
)

### Create the Query engine

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize",
    verbose=True
)

In [ ]:
# response = query_engine.query("Give me some examples of supervised learning")
#response = query_engine.query("What does the law say about violence at work")
response = query_engine.query("Do a summary of about 100 words violence at work")


1 text chunks after repacking


In [ ]:
print(response)

Violence at work is prohibited, including harassment, assault, and verbal abuse. Employers must not carry out searches on workers and are vicariously liable for violence committed by workers or third parties. Employers must investigate alleged violence and take action to protect workers within 15 days. Failure to comply can result in fines and imprisonment. Employers must also provide a safe working environment and prevent violence, including sexual harassment. Regulations may be made to enforce these provisions and ensure worker safety. Violations are considered offences with significant penalties. Worker rights are protected.
